In [33]:
from IPython.core.display import HTML
HTML("<style>.container { width:95% !important; }</style>")

# Lecture 5: Example of using available software: scipy.optimize

# Optimization software
* When we want to optimize something, we do not of course need to start everything from scratch. It is good to know how algorithms work, but if the development of new algorithms is not the main point, then one can just use packages and libraries that have been premade. 
* First, we have a look at some of the available software and, then, we have a closer look at scipy.optimize



## Have you used any optimization software before? Please share your experiences.

## Wolfram Alpha
* Free web version of Mathematica
* http://www.wolframalpha.com/
* Can perform either symbolic or numerical calculations
* Includes also some basic optimization

## Rosenbrock function
A non-convex function
$$
f(x) = (1-x_1)^2 +100(x_2-x_1^2)^2
$$
that has a global minimum in $x^*=(1,1)^T$ where $f(x^*)=0$. The minimum is located in a narrow, banana-shaped valley.

The coefficient of the second term can be adjusted but it does not affect the position of the global minimum. The Rosenbrock function is used to test optimization algorithms.


In [34]:
def f_rosenb(x):
    return (1.0 -x[0])**2 + 100*(x[1] - x[0]**2)**2

![alt text](images/rosenbrock3d.png "Rosenbrock function 3D")

![alt text](images/rosenbrock2d.png "Rosenbrock function 3D")

## Matlab - Optimization toolbox
* Interactive environment for numerical computing
* Subroutines for unconstrained optimization:
  * fminbnd: find minimum of single-variable function on fixed interval
  * fminsearch: find minimum of unconstrained multivariable function using derivative-free method
  * fminunc: find minimum of unconstrained multivariable function using gradient-based method
* https://se.mathworks.com/products/optimization.html?s_tid=srchtitle_optimization_1 
* Matlab codes for the subroutines can be found in the directory where Matlab is installed:
 ..\MATLAB\R2013a\toolbox\optim\optim\
* You can also use Octave (https://www.gnu.org/software/octave/) which is an open source software having compatibility with many Matlab scripts


# Optimization with scipy.optimize
In Python, there are multiple packages for optimization. At this lecture, we are going to take a look at *scipy.optimize* package.

## Starting up

When we want to study a package in Python, we can import it..

In [ ]:
from scipy.optimize import minimize

If we want to see the documentation, we can write the name of the package and two question marks and hit enter:

In [35]:
minimize??

or

https://docs.scipy.org/doc/scipy/tutorial/optimize.html

## Optimization of multiple variables

Let us define again our friendly objective function:

In [36]:
def f_simple(x):
    return (x[0] - 10.0)**2 + (x[1] + 5.0)**2+x[0]**2

### Method: `Nelder-Mead'

* Uses a simplex which is a shape consisting $n+1$ vertices in n dimensional space. 

* E.g., a triangle in 2D, a tetrahedron in 3D, 5-cell pentachoron (https://en.wikipedia.org/wiki/5-cell). etc.

* Begin with $n+1$ arbitrarily selected points in nD (3 in 2D) to form the simplex.
* Follow the following steps in each iteration:
    * Sort the selected points ($f1<f2<f3$)
    * Reflect the worst point ($x_r$)
    * Extend ($x_e$)
    * Contract
    * Shrink
    * Check convergency

![alt text](images/nelder_mead.png "Nelder-Mead")


The documentation has the following to say:

<pre>
    Method :ref:`Nelder-Mead <optimize.minimize-neldermead>` uses the
    Simplex algorithm [1]_, [2]_. This algorithm has been successful
    in many applications but other algorithms using the first and/or
    second derivatives information might be preferred for their better
    performances and robustness in general.
...
     References
    ----------
    .. [1] Nelder, J A, and R Mead. 1965. A Simplex Method for Function
        Minimization. The Computer Journal 7: 308-13.
    .. [2] Wright M H. 1996. Direct search methods: Once scorned, now
        respectable, in Numerical Analysis 1995: Proceedings of the 1995
        Dundee Biennial Conference in Numerical Analysis (Eds. D F
        Griffiths and G A Watson). Addison Wesley Longman, Harlow, UK.
        191-208.
</pre>

In [37]:
res = minimize(f_simple,[0,0],method='Nelder-Mead', 
         options={'disp': True})
print(res.x)
res = minimize(f_simple,[0,0],method='Powell', 
         options={'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 50.000000
         Iterations: 99
         Function evaluations: 189
[ 5.00003542 -4.99997315]
Optimization terminated successfully.
         Current function value: 50.000000
         Iterations: 2
         Function evaluations: 99
[ 5.         -5.00000004]


In [38]:
print(type(res))
print(res)
print(res.message)

<class 'scipy.optimize.optimize.OptimizeResult'>
   direc: array([[1., 0.],
       [0., 1.]])
     fun: 50.0
 message: 'Optimization terminated successfully.'
    nfev: 99
     nit: 2
  status: 0
 success: True
       x: array([ 5.        , -5.00000004])
Optimization terminated successfully.


In [39]:
res = minimize(f_rosenb,[-2.0,-10],method='Nelder-Mead', 
         options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 155
         Function evaluations: 288


In [40]:
print(res)

 final_simplex: (array([[1.00000755, 1.00001315],
       [1.00001942, 1.00004104],
       [0.99996779, 0.99993545]]), array([4.36964903e-10, 8.57224979e-10, 1.03919729e-09]))
           fun: 4.369649026757306e-10
       message: 'Optimization terminated successfully.'
          nfev: 288
           nit: 155
        status: 0
       success: True
             x: array([1.00000755, 1.00001315])


### Method: Conjugate Gradient (`CG`)
* Idea is to improve convergence properties of steepest descent
* A search direction is a combination of the current search direction and a previous search direction
* Steps in directions that are **orthogonal**
* Memory consumption 𝑂(𝑛)
* Best suited for large scale problems

![alt text](images/CG.png)

The documentation has the following to say:
<pre>
    Method :ref:`CG <optimize.minimize-cg>` uses a nonlinear conjugate
    gradient algorithm by Polak and Ribiere, a variant of the
    Fletcher-Reeves method described in [5]_ pp.  120-122. Only the
    first derivatives are used.
...
   References
    ----------
...
    .. [5] Nocedal, J, and S J Wright. 2006. Numerical Optimization.
       Springer New York.
</pre>
The Conjugate gradient method needs the gradient. The documentation has the following to say
<pre>
    jac : bool or callable, optional
        Jacobian (gradient) of objective function. Only for CG, BFGS,
        Newton-CG, L-BFGS-B, TNC, SLSQP, dogleg, trust-ncg.
        If `jac` is a Boolean and is True, `fun` is assumed to return the
        gradient along with the objective function. If False, the
        gradient will be estimated numerically.
        `jac` can also be a callable returning the gradient of the
        objective. In this case, it must accept the same arguments as `fun`.
</pre>

### Estimating the gradient numerically:

In [42]:
res = minimize(f_simple, [0,0], method='CG', #Conjugate gradient method
               options={'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 50.000000
         Iterations: 3
         Function evaluations: 21
         Gradient evaluations: 7
[ 4.99999999 -5.00000006]


### Giving the gradient with ad

In [43]:
import ad
res = minimize(f_simple, [0,0], method='CG', #Conjugate gradient method
               options={'disp': True}, jac=ad.gh(f_simple)[0])
print(res.x)

Optimization terminated successfully.
         Current function value: 50.000000
         Iterations: 3
         Function evaluations: 7
         Gradient evaluations: 7
[ 5. -5.]


### Newton-Conjugate-Gradient algorithm (method=`Newton-CG`)  

Newton-CG method uses a Newton-CG algorithm [5] pp. 168 (also known as the truncated Newton method). It uses a CG method to the compute the search direction. See also *TNC* method for a box-constrained minimization with a similar algorithm.

   References
    ----------
    .. [5] Nocedal, J, and S J Wright. 2006. Numerical Optimization.
       Springer New York.


The Newton-CG algorithm needs the Jacobian and the Hessian. The documentation has the following to say:
<pre>
    hess, hessp : callable, optional
        Hessian (matrix of second-order derivatives) of objective function or
        Hessian of objective function times an arbitrary vector p.  Only for
        Newton-CG, dogleg, trust-ncg.
        Only one of `hessp` or `hess` needs to be given.  If `hess` is
        provided, then `hessp` will be ignored.  If neither `hess` nor
        `hessp` is provided, then the Hessian product will be approximated
        using finite differences on `jac`. `hessp` must compute the Hessian
        times an arbitrary vector.
 </pre>

### Trying without reading the documentation

In [44]:
import numpy as np
res = minimize(f_simple, [0,0], method='Newton-CG', #Newton-CG method
               options={'disp': True})
print(res.x)

ValueError: Jacobian is required for Newton-CG method

### Giving the gradient

In [45]:
import ad
res = minimize(f_simple, [0,0], method='Newton-CG', #Newton-CG method
               options={'disp': True},jac=ad.gh(f_simple)[0])
print(res.x)

Optimization terminated successfully.
         Current function value: 50.000000
         Iterations: 7
         Function evaluations: 7
         Gradient evaluations: 15
         Hessian evaluations: 0
[ 4.99999999 -4.99999999]


### Giving also the hessian

In [46]:
import ad
res = minimize(f_simple, [0,0], method='Newton-CG', #Newton-CG method
               options={'disp': True},jac=ad.gh(f_simple)[0],
               hess=ad.gh(f_simple)[1])
print(res.x)

Optimization terminated successfully.
         Current function value: 50.000000
         Iterations: 7
         Function evaluations: 7
         Gradient evaluations: 7
         Hessian evaluations: 7
[ 5. -5.]


## Another example
$$
\begin{align}
\min \quad & (x_1-2)^4+(x_1 - 2x_2)^2\\
\text{s.t.}\quad &x_1,x_2\in\mathbb R
\end{align}  
$$
Optimal solution clearly is $x^*=(2,1)^T$

In [47]:
def f_simple2(x):
    return (x[0] - 2.0)**4 + (x[0] - 2.0*x[1])**2

In [48]:
import numpy as np
x0 = np.array([-96,-2000])
res = minimize(f_simple2,x0,method='Nelder-Mead', 
         options={'disp': True})
print(res.x)
res = minimize(f_simple2,x0,method='Powell', 
         options={'disp': True})
print(res.x)
res = minimize(f_simple2, x0, method='CG', #Conjugate gradient method
               options={'disp': True}, jac=ad.gh(f_simple2)[0])
print(res.x)
res = minimize(f_simple2, x0, method='Newton-CG', #Newton-CG method
               options={'disp': True},jac=ad.gh(f_simple2)[0],
               hess=ad.gh(f_simple2)[1])
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 79
         Function evaluations: 148
[2.0000004  1.00000021]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 29
         Function evaluations: 784
[1.99999649 0.99999824]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 14
         Function evaluations: 32
         Gradient evaluations: 32
[2.00005457 1.00002728]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 28
         Function evaluations: 30
         Gradient evaluations: 30
         Hessian evaluations: 28
[1.99649493 0.99824742]


In [ ]:
# Add here optimization of the Rosenbrock function with gradient based algorithms

In [49]:
x0 = np.array([-2.0,-10])
res = minimize(f_rosenb, x0, method='CG', #Conjugate gradient method
               options={'disp': True}, jac=ad.gh(f_rosenb)[0])
print(res.x)
res = minimize(f_rosenb, x0, method='Newton-CG', #Newton-CG method
               options={'disp': True},jac=ad.gh(f_rosenb)[0],
               hess=ad.gh(f_rosenb)[1])
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 30
         Function evaluations: 71
         Gradient evaluations: 71
[0.99999053 0.99998103]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 31
         Function evaluations: 36
         Gradient evaluations: 36
         Hessian evaluations: 31
[0.99999884 0.99999768]


## Line search

In [50]:
def f_singlevar(x):
    return 2+(1-x)**2

In [51]:
from scipy.optimize import minimize_scalar
minimize_scalar??

### Method: `Golden`

The documentation has the following to say:
<pre>
    Method :ref:`Golden <optimize.minimize_scalar-golden>` uses the
    golden section search technique. It uses analog of the bisection
    method to decrease the bracketed interval. It is usually
    preferable to use the *Brent* method.
</pre>

In [52]:
minimize_scalar(f_singlevar,method='golden',tol=0.00001)

     fun: 2.0
    nfev: 30
     nit: 25
 success: True
       x: 1.0

### Method: `Brent`

The documentation has the following to say about the Brent method:

    Method *Brent* uses Brent's algorithm to find a local minimum.
    The algorithm uses inverse parabolic interpolation when possible to
    speed up convergence of the golden section method.

In [53]:
minimize_scalar(f_singlevar,method='brent')

     fun: 2.0
    nfev: 8
     nit: 4
 success: True
       x: 0.99999998519